In [10]:
import pandas as pd 
import numpy as np
import pyblp
from sklearn.linear_model import LinearRegression

In [11]:
product_data = pd.read_csv('../data/market_integrates_100.csv')
print(product_data)

     market_ids  firm_ids  characteristic1  characteristic2      prices  \
0             1         1         5.433865         1.380927   28.763676   
1             1         2         5.907737         4.714381   34.087701   
2             1         3         5.943911         2.569512   29.428713   
3             1         4         5.329441         8.604975   46.868650   
4             1         5         5.083033         2.233709   31.553979   
..          ...       ...              ...              ...         ...   
995         100         6         5.360283         6.225416   40.387160   
996         100         7         5.958183         6.851203   41.990144   
997         100         8         5.184745         4.121598   40.690043   
998         100         9         5.223118         5.184629   43.004231   
999         100        10         5.019838         7.083520  556.469894   

     marginal_cost        shares       profits       markups    e_quantity  \
0     2.123684e+01  5

In [12]:
# Getting the dataset estimation ready with at least a supply_side instrument/ demand_side instrument 
product_data_complete = product_data.rename(columns={'labor':'demand_instrument0', 'capital':'demand_instruments1'})
print(product_data_complete)



     market_ids  firm_ids  characteristic1  characteristic2      prices  \
0             1         1         5.433865         1.380927   28.763676   
1             1         2         5.907737         4.714381   34.087701   
2             1         3         5.943911         2.569512   29.428713   
3             1         4         5.329441         8.604975   46.868650   
4             1         5         5.083033         2.233709   31.553979   
..          ...       ...              ...              ...         ...   
995         100         6         5.360283         6.225416   40.387160   
996         100         7         5.958183         6.851203   41.990144   
997         100         8         5.184745         4.121598   40.690043   
998         100         9         5.223118         5.184629   43.004231   
999         100        10         5.019838         7.083520  556.469894   

     marginal_cost        shares       profits       markups    e_quantity  \
0     2.123684e+01  5

In [13]:
# the endog regressor
X1_formulation = pyblp.Formulation('1 + prices + characteristic1 + characteristic2')

# the only regressor on which we want random coeff 
X2_formulation = pyblp.Formulation('0 +  prices')

product_formulations = (X1_formulation, X2_formulation)
print(product_formulations)

(1 + prices + characteristic1 + characteristic2, prices)


In [14]:
mc_integration = pyblp.Integration('monte_carlo', size=50, specification_options={'seed': 0})


In [15]:
pr_integration = pyblp.Integration('product', size=5)


In [16]:
mc_problem = pyblp.Problem(product_formulations, product_data_complete, integration=mc_integration)
print(mc_problem)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F    I     K1    K2    MD 
---  ----  ---  ----  ----  ----  ----
100  1000  10   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          
Dimensions:
 T    N     F    I     K1    K2    MD 
---  ----  ---  ----  ----  ----  ----
100  1000  10   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          


'demand_instruments1' was specified but not 'demand_instruments0'.


In [17]:
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})


In [18]:
results1 = mc_problem.solve(sigma=np.ones((1, 1)), optimization=bfgs)


Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:     prices    
------  -------------
prices  +1.000000E+00
Starting optimization ...

GMM   Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Gradient                  
Step   Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement       Norm           Theta    
----  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  -------------
 1         0             1          7211         21753        9     +4.043380E+12                 +6.439616E+07  +1.000000E+00

At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If the errors persist or seem to be impacting the optimization results, consider setting an error punishment or following any of the other suggestions below:
The fi